In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List

from tqdm import tqdm

from spluslib import SplusService, ImageType
from xmatchlib import XTable, CrossMatch
from astromodule.io import load_table, save_table
from astromodule.tableops import stilts_crossmatch, stilts_unique
from astromodule.legacy import LegacyService
from astropy import units as u

# Prepare Dataset

In [2]:
df_part1_raw = load_table('tables/catalog_v4_part1+raw.csv', comment='#')
df_part2_raw = load_table('tables/catalog_v4_part2+raw.csv', comment='#')
df_part3_raw = load_table('tables/catalog_v4_part3+raw.csv', comment='#')
df_part3_allow = load_table('tables/catalog_v4_part3+allow.csv', comment='#')
df_part4_raw = load_table('tables/catalog_v4_part4+raw.csv', comment='#')
df_part4_deny = load_table('tables/catalog_v4_part4+deny.csv', comment='#')
df_part5_raw = load_table('tables/catalog_v5_part5+raw.csv', comment='#')
df_v5_deny = load_table('tables/catalog_v4_deny.csv', comment='#')

In [3]:
columns = ['cluster_name', 'ra', 'dec', 'z']

df_part1 = df_part1_raw[columns]
df_part1['shard'] = [1] *len(df_part1)
df_part1['release'] = ['dr4'] * len(df_part1)

df_part2 = df_part2_raw[columns]
df_part2['shard'] = [2] *len(df_part2)
df_part2['release'] = ['dr4'] * len(df_part2)

df_part3 = df_part3_raw[df_part3_raw.cluster_name.isin(df_part3_allow.cluster_name.values)]
df_part3 = df_part3[columns]
df_part3['shard'] = [3] *len(df_part3)
df_part3['release'] = ['dr4'] * len(df_part3)

df_part4 = df_part4_raw[~df_part4_raw.cluster_name.isin(df_part4_deny.cluster_name.values)]
df_part4 = df_part4[columns]
df_part4['shard'] = [4] *len(df_part4)
df_part4['release'] = ['idr5'] * len(df_part4)

df_part5 = df_part5_raw[columns]
df_part5['release'] = ['dr4'] * len(df_part5)

print('#PART_3_ALLOW:', len(df_part3_allow), ' |  #PART_3:', len(df_part3))
print('#PART_4_RAW - #PART_4_DENY:', len(df_part4_raw) - len(df_part4_deny), ' |  #PART_4:', len(df_part4))

#PART_3_ALLOW: 42  |  #PART_3: 31
#PART_4_RAW - #PART_4_DENY: 21  |  #PART_4: 22


### Concatenação dos pedaços

In [4]:
df_v5_raw = pd.concat((df_part1, df_part2, df_part3, df_part4, df_part5))
print('len:', len(df_v5_raw), ' | ', 'columns:', df_v5_raw.columns.tolist())
save_table(df_v5_raw, 'tables/catalog_v5_raw.csv')

len: 155  |  columns: ['cluster_name', 'ra', 'dec', 'z', 'shard', 'release']


### Padronização dos nomes e remoção de nomes duplicados

In [5]:
df_v5 = df_v5_raw.copy()
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(' ', '')
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^ABELL0+', 'A', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^ABELL', 'A', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^Abell0+', 'A', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^Abell', 'A', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^A0+', 'A', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^AS', 'A', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^ACO', 'A', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^A(\d+)\w*', r'A\1', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^RXC', 'MCXC', regex=True)
df_v5['cluster_name'] = df_v5.cluster_name.str.replace(r'^MKW0+', 'MKW', regex=True)
df_v5 = df_v5.drop_duplicates('cluster_name', keep='last')

print('len:', len(df_v5))

len: 116


### Crossmatch para identificação de duplicados

In [6]:
df_v5_unique = stilts_unique(
  table=df_v5,
  radius=45*u.arcmin,
  action='identify'
)
df_v5_unique[df_v5_unique.GroupID > 0].sort_values('GroupID')

,cluster_name,ra,dec,z,shard,release,GroupID,GroupSize
0,A194,21.420000,-1.407200,0.018000,2.0,dr4,1,2
56,MCXCJ0125.6-0125,21.387800,-1.506900,0.018000,3.0,dr4,1,2
28,WHYJ034138-531028,55.412308,-53.180950,0.085200,2.0,dr4,2,2
29,A3158,55.727288,-53.640609,0.059700,2.0,dr4,2,2
66,NGC3258,157.148000,-35.642000,0.009300,3.0,dr4,3,3
114,Antlia,157.421232,-35.325808,0.011054,NaN,dr4,3,3
38,PSZ2G272.88+19.14,157.429700,-35.330424,0.011100,2.0,dr4,3,3
57,HCG048,159.440200,-27.080500,0.009000,3.0,dr4,4,2
51,Hydra,159.174160,-27.524440,0.012600,2.0,dr4,4,2
59,MCXCJ1253.2-1522,193.310000,-15.380000,0.046200,3.0,dr4,5,2


In [7]:
df_v5 = df_v5[~df_v5.cluster_name.isin(df_v5_deny.cluster_name)]
print('len:', len(df_v5))

len: 82


### Criação dos indices

In [8]:
if 'cluster_id' not in df_v5.columns:
  df_v5.insert(0, 'cluster_id', range(1, len(df_v5) + 1))
save_table(df_v5, 'tables/catalog_v5.csv')
save_table(df_v5.sort_values('ra'), 'tables/catalog_v5_RA.csv')
save_table(df_v5.sort_values('dec'), 'tables/catalog_v5_DEC.csv')
df_v5

,cluster_id,cluster_name,ra,dec,z,shard,release
0,1,A194,21.420000,-1.407200,0.018000,2.0,dr4
1,2,A2457,338.920000,1.484900,0.057800,2.0,dr4
2,3,A267,28.174850,1.007090,0.232700,2.0,dr4
3,4,MCXCJ0458.9-0029,74.729580,-0.489170,0.015000,2.0,dr4
4,5,MCXCJ0920.0+0102,140.002080,1.040000,0.017500,2.0,dr4
...,...,...,...,...,...,...,...
4,78,A3716,312.819583,-52.695278,0.047000,NaN,dr4
5,79,MZ00407,329.633427,-28.616715,0.021682,NaN,dr4
6,80,A147,17.041667,2.166667,0.043800,NaN,dr4
7,81,Antlia,157.421232,-35.325808,0.011054,NaN,dr4


In [19]:
def compute_radius(z):
  df_radius = pd.read_csv('tables/z_rad15mpc-degb.dat', sep=' ')
  return df_radius.iloc[(df_radius['z'] - z).abs().argsort()[:1]]['radius'].values[0]


def batch_download_cluster_region(
  clusters: List[str], 
  central_ra:List[float], 
  central_dec: List[float], 
  radius: List[float], 
):
  query_template = """
  SELECT 
    dual_g.RA, dual_g.DEC, dual_g.Field, 
    dual_g.g_auto, dual_r.r_auto, dual_i.i_auto, dual_u.u_auto, dual_z.z_auto, dual_r.r_aper_6,
    photoz.zml, photoz.odds, '{cluster}' AS cluster
  FROM
    dr4_dual_g AS dual_g
    INNER JOIN
    dr4_dual_i AS dual_i ON dual_i.ID = dual_g.ID
    INNER JOIN
    dr4_dual_r AS dual_r ON dual_r.ID = dual_g.ID
    INNER JOIN
    dr4_dual_u AS dual_u ON dual_u.ID = dual_g.ID
    INNER JOIN
    dr4_dual_z AS dual_z ON dual_z.ID = dual_g.ID
    INNER JOIN
    dr4_vacs.dr4_gal_photoz AS photoz ON photoz.ID = dual_g.ID
  WHERE
    r_auto BETWEEN {r_min} AND {r_max} AND
    1 = CONTAINS(
      POINT('ICRS', dual_g.RA, dual_g.DEC), 
      CIRCLE('ICRS', {ra}, {dec}, {radius})
    )
  """
  
  mag_ranges = [
    (13, 15), (15, 16), (16, 17), (17, 17.5), (17.5, 18), (18, 18.5), 
    (18.5, 19), (19, 19.5), (19.5, 20), (20, 20.5), (20.5, 21), (21, 21.5),
  ]
  
  for i in range(len(clusters)):
    _cluster = clusters[i]
    _central_ra = central_ra[i]
    _central_dec = central_dec[i]
    _radius = radius[i]
    
    print(f'[{i + 1} / {len(clusters)}] {_cluster}')
    
    save_path = Path('outputs_v5') / 'photo' / f'cluster_{_cluster}.csv'
    
    if save_path.exists(): continue
    
    query = [
      query_template.format(
        cluster=_cluster, 
        ra=_central_ra, 
        dec=_central_dec, 
        radius=_radius,
        r_min=r_min,
        r_max=r_max,
      )
      for (r_min, r_max) in mag_ranges
    ]

    # save_path = [
    #   Path('outputs_v5') / f'cluster_{_cluster}.csv' for _cluster in clusters
    # ]

    sp = SplusService(username='natanael', password='natan')
    sp.batch_query(query, save_path=save_path, replace=True, join=True, workers=5)


def batch_search_idr5(df_photo, df_clusters, replace=False):
  output_folder = Path('outputs_v5')
  columns = ['RA', 'DEC', 'zml', 'odds', 'field']
  for i, cluster in (pbar := tqdm(df_clusters.iterrows(), total=len(df_clusters))):
    z = cluster['z']
    ra = cluster['ra']
    dec = cluster['dec']
    name = cluster['cluster_name']
    save_path = output_folder / 'photo' / f'cluster_{name}.csv'
    search_radius = compute_radius(z)
    pbar.set_description(f'{name} ({search_radius:.2f} deg)')
    if save_path.exists() and not replace: continue
    search_radius2 = search_radius ** 2
    df = df_photo[
      (((df_photo.RA - ra)**2 + (df_photo.DEC - dec)**2) < search_radius2)
    ]
    save_table(df[columns], save_path)



def batch_search_spec(df_spec, df_clusters, suffix, replace=False):
  Z_RANGE = 0.02
  output_folder = Path('outputs_v5')
  columns = ['RA', 'DEC', 'z', 'e_z']
  for i, cluster in (pbar := tqdm(df_clusters.iterrows(), total=len(df_clusters))):
    z = cluster['z']
    ra = cluster['ra']
    dec = cluster['dec']
    name = cluster['cluster_name']
    save_path = output_folder / 'spec' / f'{name}_{suffix}.parquet'
    search_radius = compute_radius(z)
    pbar.set_description(f'{name} ({search_radius:.2f} deg)')
    if save_path.exists() and not replace: continue
    search_radius2 = search_radius ** 2
    df = df_spec[
      (((df_spec.RA - ra)**2 + (df_spec.DEC - dec)**2) < search_radius2) &
      df_spec['z'].between(z - Z_RANGE, z + Z_RANGE) &
      df_spec['f_z'].str.startswith('KEEP')
    ]
    save_table(df[columns], save_path)
    
    
    
def batch_crossmatch_spec_photo(df_clusters, variant, replace=False):
  output_folder = Path('outputs_v5')
  for i, cluster in (pbar := tqdm(df_clusters.iterrows(), total=len(df_clusters))):
    name = cluster['cluster_name']
    save_path = output_folder / 'match' / f'{name}_match+{variant}.parquet'
    pbar.set_description(f'{name}')
    if save_path.exists() and not replace: continue
    df_spec = output_folder / 'spec' / f'{name}_spec+{variant}.parquet'
    df_photo = output_folder / 'photo' / f'cluster_{name}.csv'
    df = stilts_crossmatch(
      table1=df_spec,
      table2=df_photo,
      ra1='RA',
      dec1='DEC',
      ra2='RA',
      dec2='DEC',
      join='all1',
      radius=1*u.arcsec,
    )
    save_table(df, save_path)
    


def batch_legacy_crossmatch(df_clusters, variant, replace=False):
  output_folder = Path('outputs_v5')
  for i, cluster in df_clusters.iterrows():
    name = cluster['cluster_name']
    save_path = output_folder / 'leg' / f'{name}_leg+{variant}.parquet'
    leg_inner_path = output_folder / 'leg' / f'{name}_leg_inner+all.parquet'
    df_match = output_folder / 'match' / f'{name}_match+{variant}.parquet'
    print(f'[{i+1} / {len(df_clusters)}] {name}')
    if variant == 'all':
      if not (leg_inner_path.exists() and not replace):
        ls = LegacyService()
        ls.crossmatch(
          df_match, 
          columns=['ra', 'dec', 'mag_r', 'mag_g', 'mag_i', 'mag_z'], 
          radius=1*u.arcsec, 
          save_path=leg_inner_path,
          workers=4,
        )
    if True:#not (save_path.exists() and not replace):
      df = stilts_crossmatch(
        table1=df_match,
        table2=leg_inner_path,
        ra1='RA_1',
        dec1='DEC_1',
        ra2='ra',
        dec2='dec',
        join='all1',
        radius=1*u.arcsec,
        fmt='csv',
      )
      if cluster['release'] == 'dr4':
        columns = [
          'RA_1', 'DEC_1', 'z', 'e_z', 'zml', 'odds', 'g_auto',
          'r_auto', 'i_auto', 'u_auto', 'z_auto', 'r_aper_6', 'cluster', 
          'mag_r', 'mag_g', 'mag_i', 'mag_z'
       ]
      else:
        columns = [
          'RA_1', 'DEC_1', 'z', 'e_z', 'zml', 'odds', 
          'mag_r', 'mag_g', 'mag_i', 'mag_z'
        ]
      df = df[columns]
      df = df.rename(columns={
        'mag_r': 'ls10_r', 
        'mag_g': 'ls10_g', 
        'mag_i': 'ls10_i', 
        'mag_z': 'ls10_z'
      })
      df['ls10_photo'] = (
        ~df.ls10_r.isna() & ~df.ls10_g.isna() & 
        ~df.ls10_i.isna() & ~df.ls10_z.isna()
      )
      df['ls10_photo'] = df['ls10_photo'].astype(int)
      save_table(df, save_path)


    
    
def batch_prepare_tables(df_clusters, variant, replace=False):
  output_folder = Path('outputs_v5') / 'paulo' / variant / 'clusters'
  output_folder.mkdir(exist_ok=True, parents=True)
  for i, row in (pbar := tqdm(df_clusters.iterrows(), total=len(df_clusters))):
    name = row['cluster_name']
    clsid = row['cluster_id']
    release = row['release']
    pbar.set_description(name)
    save_path = output_folder / f'cluster_{str(clsid).zfill(4)}.dat'
    if save_path.exists() and not replace: continue
    if release == 'idr5':
      columns = [
        'RA_1', 'DEC_1', 'z', 'e_z', 'zml', 'odds', 
        'ls10_g', 'ls10_r', 'ls10_i', 'ls10_z', 'ls10_photo'
      ]
      rename_map = {
        'RA_1': 'RA', 
        'DEC_1': 'DEC',
        'z': 'zspec',
        'e_z': 'zspec-err',
        'zml': 'z_phot', 
        'odds': 'z_phot_odds',
      }
      df = load_table(Path('outputs_v5') / 'leg' / f'{name}_leg+{variant}.parquet')
      df = df[columns]
      df = df.rename(columns=rename_map)
      df = df.fillna(-999)
      df['splus_g'] = [-999] * len(df)
      df['splus_r'] = [-999] * len(df)
      df['splus_i'] = [-999] * len(df)
      df['splus_u'] = [-999] * len(df)
      df['splus_z'] = [-999] * len(df)
    elif release == 'dr4':
      columns = [
        'RA_1', 'DEC_1', 'z', 'e_z', 'zml', 'odds', 
        'ls10_g', 'ls10_r', 'ls10_i', 'ls10_z', 'ls10_photo',
        'g_auto', 'r_auto', 'i_auto', 'u_auto', 'z_auto'
      ]
      rename_map = {
        'RA_1': 'RA', 
        'DEC_1': 'DEC',
        'z': 'zspec',
        'e_z': 'zspec-err',
        'zml': 'z_phot', 
        'odds': 'z_phot_odds',
        'g_auto': 'splus_g', 
        'r_auto': 'splus_r', 
        'i_auto': 'splus_i', 
        'u_auto': 'splus_u', 
        'z_auto': 'splus_z',
      }
      df = load_table(Path('outputs_v5') / 'leg' / f'{name}_leg+{variant}.parquet')
      df = df[columns]
      df = df.rename(columns=rename_map)
      df = df.fillna(-999)
    save_table(df, save_path)
  index_columns = ['cluster_id', 'cluster_name', 'ra', 'dec', 'z']
  index_rename = {
    'cluster_name': 'name',
    'cluster_id': 'clsid',
    'z': 'zspec',
    'ra': 'RA',
    'dec': 'DEC'
  }
  df_index = df_clusters[index_columns].rename(columns=index_rename)
  save_table(df_index, output_folder.parent / 'index.dat')
      



def concat_tables(paths, save_path):
  df = load_table(paths[0])
  
  for i in tqdm(range(1, len(paths))):
    df2 = pd.read_csv(paths[i])
    df = pd.concat((df, df2), ignore_index=True)
  
  df = df[df.columns.drop(list(df.filter(regex='Unnamed:*')))]
  save_table(df, save_path)

In [11]:
df_clusters = load_table('tables/catalog_v5.csv')
df_clusters = df_clusters[df_clusters.release=='dr4']
clusters = df_clusters['cluster_name'].to_list()
central_ra = df_clusters['ra'].to_list()
central_dec = df_clusters['dec'].to_list()
radius = [compute_radius(_z) for _z in df_clusters['z'].to_list()]
batch_download_cluster_region(
  clusters=clusters,
  central_ra=central_ra,
  central_dec=central_dec,
  radius=radius
)

[1 / 67] A194
[2 / 67] A2457
[3 / 67] A267
[4 / 67] MCXCJ0458.9-0029
[5 / 67] MCXCJ0920.0+0102
[6 / 67] HCG97
[7 / 67] CL1058+0137
[8 / 67] MCXCJ1121.7+0249
[9 / 67] IC1365
[10 / 67] ESO351-021
[11 / 67] A119
[12 / 67] A168
[13 / 67] MS0116.3-0115
[14 / 67] WHLJ012023-000444
[15 / 67] MCXCJ0229.3-3332
[16 / 67] NGC1132
[17 / 67] A3112
[18 / 67] A3122
[19 / 67] A3135
[20 / 67] Fornax
[21 / 67] MCXCJ0340-4542
[22 / 67] A3158
[23 / 67] A3223
[24 / 67] MCXCJ0413.9-3805
[25 / 67] A463
[26 / 67] A970
[27 / 67] MKW4
[28 / 67] A1644
[29 / 67] NGC5004
[30 / 67] A3733
[31 / 67] A3744
[32 / 67] RXJ2137.1+0026
[33 / 67] A2440
[34 / 67] A3880
[35 / 67] Hydra
[36 / 67] A3266
[37 / 67] A4038
[38 / 67] A3581
[39 / 67] A3128
[40 / 67] A1631
[41 / 67] WBL074
[42 / 67] A1750
[43 / 67] A4049
[44 / 67] NGC3258
[45 / 67] WBL288
[46 / 67] MKW1
[47 / 67] [YMV2007]3820
[48 / 67] [YMV2007]49
[49 / 67] [YMV2007]4713
[50 / 67] WBL550
[51 / 67] [YMV2007]7604
[52 / 67] [YMV2007]7609
[53 / 67] WBL081
[54 / 67] A301


100%|██████████| 12/12 [13:18<00:00, 66.50s/ files]


[67 / 67] IIZw108


100%|██████████| 12/12 [03:48<00:00, 19.03s/ files]


In [12]:
df_clusters = load_table('tables/catalog_v5.csv')
df_clusters = df_clusters[df_clusters.release=='idr5']
df_photo = load_table('/mnt/hd/natanael/astrodata/idr5_photoz/idr5_photoz.parquet')
batch_search_idr5(
  df_clusters=df_clusters,
  df_photo=df_photo,
  replace=False,
)

A4059 (4.37 deg): 100%|██████████| 15/15 [00:00<00:00, 142.76it/s]


In [13]:
df_clusters = load_table('tables/catalog_v5.csv')
df_spec = load_table('tables/SpecZ_Catalogue_20231101.parquet')
df_spec_GU = df_spec[
  df_spec.class_spec.str.startswith('GALAXY') | 
  df_spec.class_spec.str.startswith('UNCLEAR')
].copy(deep=True)
df_spec_G = df_spec_GU[
  df_spec_GU.class_spec.str.startswith('GALAXY')
].copy(deep=True)
print('SPEC ALL')
batch_search_spec(
  df_spec=df_spec,
  df_clusters=df_clusters,
  suffix='spec+all',
  replace=False,
)
print('SPEC GALAXY + UNCLEAR')
batch_search_spec(
  df_spec=df_spec_GU,
  df_clusters=df_clusters,
  suffix='spec+G+U',
  replace=False,
)
print('SPEC GALAXY')
batch_search_spec(
  df_spec=df_spec_G,
  df_clusters=df_clusters,
  suffix='spec+G',
  replace=False,
)

SPEC ALL


IIZw108 (4.35 deg): 100%|██████████| 82/82 [00:18<00:00,  4.51it/s]           


SPEC GALAXY + UNCLEAR


IIZw108 (4.35 deg): 100%|██████████| 82/82 [00:11<00:00,  7.32it/s]          


SPEC GALAXY


IIZw108 (4.35 deg): 100%|██████████| 82/82 [00:07<00:00, 10.80it/s]           


In [14]:
df_clusters = load_table('tables/catalog_v5.csv')
print('Match ALL')
batch_crossmatch_spec_photo(
  df_clusters=df_clusters,
  variant='all',
  replace=False,
)
print('Match G+U')
batch_crossmatch_spec_photo(
  df_clusters=df_clusters,
  variant='G+U',
  replace=False,
)
print('Match G')
batch_crossmatch_spec_photo(
  df_clusters=df_clusters,
  variant='G',
  replace=False,
)

Match ALL


IIZw108: 100%|██████████| 82/82 [01:25<00:00,  1.05s/it]       


Match G+U


IIZw108: 100%|██████████| 82/82 [01:17<00:00,  1.06it/s]       


Match G


IIZw108: 100%|██████████| 82/82 [01:17<00:00,  1.06it/s]       


In [20]:
df_clusters = load_table('tables/catalog_v5.csv')
print('Legacy Match ALL')
batch_legacy_crossmatch(
  df_clusters=df_clusters,
  variant='all',
  replace=False
)
print('Legacy Match G+U')
batch_legacy_crossmatch(
  df_clusters=df_clusters,
  variant='G+U',
  replace=False
)
print('Legacy Match G')
batch_legacy_crossmatch(
  df_clusters=df_clusters,
  variant='G',
  replace=False
)

Legacy Match ALL
[1 / 82] A194
[2 / 82] A2457
[3 / 82] A267
[4 / 82] MCXCJ0458.9-0029
[5 / 82] MCXCJ0920.0+0102
[6 / 82] HCG97
[7 / 82] CL1058+0137
[8 / 82] MCXCJ1121.7+0249
[9 / 82] IC1365
[10 / 82] ESO351-021
[11 / 82] A119
[12 / 82] A168
[13 / 82] MS0116.3-0115
[14 / 82] WHLJ012023-000444
[15 / 82] MCXCJ0229.3-3332
[16 / 82] NGC1132
[17 / 82] A3112
[18 / 82] A3122
[19 / 82] A3135
[20 / 82] Fornax
[21 / 82] MCXCJ0340-4542
[22 / 82] A3158
[23 / 82] A3223
[24 / 82] MCXCJ0413.9-3805
[25 / 82] A463
[26 / 82] A970
[27 / 82] MKW4
[28 / 82] A1644
[29 / 82] NGC5004
[30 / 82] A3733
[31 / 82] A3744
[32 / 82] RXJ2137.1+0026
[33 / 82] A2440
[34 / 82] A3880
[35 / 82] Hydra
[36 / 82] A3266
[37 / 82] A4038
[38 / 82] A3581
[39 / 82] A3128
[40 / 82] A1631
[41 / 82] WBL074
[42 / 82] A1750
[43 / 82] A4049
[44 / 82] NGC3258
[45 / 82] WBL288
[46 / 82] MKW1
[47 / 82] [YMV2007]3820
[48 / 82] [YMV2007]49
[49 / 82] [YMV2007]4713
[50 / 82] WBL550
[51 / 82] [YMV2007]7604
[52 / 82] [YMV2007]7609
[53 / 82] WBL08

In [23]:
df_clusters = load_table('tables/catalog_v5.csv')
batch_prepare_tables(
  df_clusters=df_clusters,
  variant='all',
  replace=True,
)
batch_prepare_tables(
  df_clusters=df_clusters,
  variant='G',
  replace=True,
)
batch_prepare_tables(
  df_clusters=df_clusters,
  variant='G+U',
  replace=True,
)

IIZw108: 100%|██████████| 82/82 [00:03<00:00, 26.06it/s]          


In [24]:
for path in (Path('outputs_v5') / 'paulo' / 'G+U' / 'clusters').glob('*.dat'):
  df = load_table(path)
  print(f'{path.stem.split("_")[1]}', end=' ')
  print(*df.columns, sep=' | ')

0057 RA | DEC | zspec | zspec-err | z_phot | z_phot_odds | ls10_g | ls10_r | ls10_i | ls10_z | ls10_photo | splus_g | splus_r | splus_i | splus_u | splus_z
0013 RA | DEC | zspec | zspec-err | z_phot | z_phot_odds | ls10_g | ls10_r | ls10_i | ls10_z | ls10_photo | splus_g | splus_r | splus_i | splus_u | splus_z
0076 RA | DEC | zspec | zspec-err | z_phot | z_phot_odds | ls10_g | ls10_r | ls10_i | ls10_z | ls10_photo | splus_g | splus_r | splus_i | splus_u | splus_z
0067 RA | DEC | zspec | zspec-err | z_phot | z_phot_odds | ls10_g | ls10_r | ls10_i | ls10_z | ls10_photo | splus_g | splus_r | splus_i | splus_u | splus_z
0040 RA | DEC | zspec | zspec-err | z_phot | z_phot_odds | ls10_g | ls10_r | ls10_i | ls10_z | ls10_photo | splus_g | splus_r | splus_i | splus_u | splus_z
0043 RA | DEC | zspec | zspec-err | z_phot | z_phot_odds | ls10_g | ls10_r | ls10_i | ls10_z | ls10_photo | splus_g | splus_r | splus_i | splus_u | splus_z
0077 RA | DEC | zspec | zspec-err | z_phot | z_phot_odds | ls10_